# Load the MLFlow model locally and try predictions

## Prerequisites

1. You need to have run successfully the training notebook related to this model, available in this same folder, where at the end of the notebook, after training the model, it downloads the 'artifacts' with the MLFlow model folder ("./artifact_downloads/outputs/mlflow-model").

2. Create a conda environment with the 'conda.yaml' file provided within the "mlflow-model" folder, doing like the following:

   1. (base) /> conda env create --file conda.yaml --name automl-model-classification-env
   
   1. (base) /> conda activate automl-model-classification-env
   
   1. (automl-model-classification-env) /> conda install jupyter nb_conda

3. Run Jupyter and make sure you are using the related 'automl-model-classif-env' Kernel.

4. Run this notebook.

If the MLFlow model files were downloaded successfully by the training notebook, you should see the files here.

In [ ]:
import os

# Show the contents of the MLFlow model folder
os.listdir("./artifact_downloads/outputs/mlflow-model")

# You should see a list of files such as the following:
# ['conda.yaml', 'MLmodel', 'model.pkl', 'requirements.txt']

### Load the test data into a Pandas DataFrame

Load a test dataset (bank_marketing_test_data.csv) into a Pandas DataFrame in order to try some predictions with it.

In [ ]:
import pandas as pd

test_df = pd.read_csv(
    "./data/test-mltable-folder/nyc_energy_test_clean.csv"
).reset_index(drop=True)
y_actual = test_df.pop("demand").values

test_df.shape, y_actual.shape

To use TabularDataset created in V1 sdk as a test data on the batch end point inference we need to convert it to V2 Input.

In [ ]:
"""
from mltable import load
filedataset_asset = ml_client.data.get(name="nyc_energy_test",version=1)
test_df = load(f"azureml:/{filedataset_asset.id}").to_pandas_dataframe().reset_index(drop=True)
y_actual = test_df.pop("demand").values

test_df.shape, y_actual.shape
"""

## Load the best model in memory

Load the model using MLflow flavor. Check MLmodel under the downloaded folder (artifact_downloads/outputs/mlflow-model).
For this particular example (and for AutoML tabular scenario except for forecast tcn which has Pytorch flavor), MLmodel file will describe two flavors, sklearn and python_function.
We show how to load model using sklearn for forecast models, which predict with .forecast() and .forecast_quantiles().
For more information on MLflow flavors, visit: https://www.mlflow.org/docs/latest/models.html#storage-format

Loading the models locally assume that you are running the notebook in an environment compatible with the model. The list of dependencies that is expected by the model is specified in the MLflow model produced by AutoML (in the 'conda.yaml' file within the mlflow-model folder).

Load the model using MLFlow PyFunc:

In [ ]:
import mlflow.pyfunc
import mlflow.sklearn

In [ ]:
# Get the MLFlow model from the downloaded MLFlow model files

model = mlflow.sklearn.load_model("./artifact_downloads/outputs/mlflow-model")

In [ ]:
# Forecasting models predict with .forecast() or .forecast_quantiles(), not with .predict()

y_preds = model.forecast(test_df, ignore_data_errors=True)

y_preds

# Original forecasting with .forecast_quantiles(X_test)
# https://github.com/Azure/azureml-examples/blob/main/python-sdk/tutorials/automl-with-azureml/forecasting-energy-demand/forecasting_script.py

# Next Steps
You can see further examples of other AutoML tasks such as Image-Classification, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.